In [3]:
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass

# Función para conectar con la base de datos
def connect_to_sakila(user, password, host, database):
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
    return engine

# Función rentals_month para recuperar datos de alquiler para un mes y año específicos
def rentals_month(engine, month, year):
    query = f'''
    SELECT r.rental_id, r.customer_id, r.rental_date
    FROM rental r
    WHERE MONTH(r.rental_date) = {month} AND YEAR(r.rental_date) = {year};
    '''
    with engine.connect() as connection:
        rentals_df = pd.read_sql(query, connection)
    return rentals_df

# Función rental_count_month para agrupar por customer_id y contar el número de alquileres
def rental_count_month(rentals_df):
    # Agrupar por customer_id y contar los alquileres
    rental_counts = rentals_df.groupby('customer_id').size().reset_index(name='rentals')
    return rental_counts

# Función compare_rentals para comparar el número de alquileres entre dos meses y años
def compare_rentals(engine, month1, year1, month2, year2):
    rentals1 = rentals_month(engine, month1, year1)
    rentals2 = rentals_month(engine, month2, year2)

    rental_counts1 = rental_count_month(rentals1)
    rental_counts2 = rental_count_month(rentals2)

    # Merge los DataFrames por customer_id
    comparison_df = pd.merge(rental_counts1, rental_counts2, on='customer_id', how='outer', suffixes=('_month1', '_month2'))
    
    # Calcular la diferencia y manejar valores nulos
    comparison_df['difference'] = comparison_df['rentals_month2'].fillna(0) - comparison_df['rentals_month1'].fillna(0)
    
    return comparison_df


